In [0]:
!wget "http://datasets.d2.mpi-inf.mpg.de/mateusz14visual-turing/nyu_depth_images.tar" 

In [0]:
!wget "https://datasets.d2.mpi-inf.mpg.de/mateusz14visual-turing/qa.894.raw.txt"

In [0]:
!wget "http://nlp.stanford.edu/data/glove.6B.zip"

In [0]:
!unzip glove.6B.zip

In [0]:
!tar -xvf nyu_depth_images.tar

In [0]:
from torch import nn
from torch.autograd import Variable
import torch
import numpy as np
import glob
import os
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
import torch.cuda as cuda
import torch.utils.data as torchdata
import torchvision.models as models
import torch.nn.functional as F
import random
from torch.utils.data.sampler import SubsetRandomSampler
import copy

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [0]:


glove_path = '/content/glove.6B.300d.txt'

#create word dictionary
def load_glove(path):
  with open(path) as f:
    glove = {}
    for line in f.readlines():
      values = line.split()
      word = values[0]
      vector = np.array(values[1:], dtype='float32')
      glove[word] = vector
    return glove


glove = load_glove(glove_path)

In [0]:
def create_emb_layer(weights_matrix, non_trainable= False):
  num_embeddings, embedding_dim = weights_matrix.shape
  emb_layer = nn.Embedding(num_embeddings, embedding_dim)
  emb_layer.load_state_dict({'weight': torch.from_numpy(weights_matrix)})
  if non_trainable:
    emb_layer.weight.requires_grad = False
  return emb_layer, num_embeddings, embedding_dim

In [0]:
my_vocab = set()
all_questions = []
all_answers = []
with open('qa.894.raw.txt') as f:
  lines = f.readlines()
  for line in lines:
    vocab = list(line.split())
    vocab = [x.replace(',','') for x in vocab]
    if any('?' in s for s in vocab):
      temp = copy.deepcopy(vocab)
      all_questions.append(temp)
    else:
      temp = copy.deepcopy(vocab)
      all_answers.append(temp)
    for i in range(len(vocab)):
      if 'image' in vocab[i]:
        vocab[i] = 'image'
    my_vocab.update(vocab)
    


In [0]:
matrix_len = len(my_vocab)
weights_matrix = np.zeros((matrix_len, 300))
words_found = 0
final_vocab = dict()
for i, word in enumerate(my_vocab):
  try:
    if '_' in word:
      temp = copy.deepcopy(word)
      temp = temp.split('_')
      try:
        s1 = glove[temp[0]]
      except KeyError:
        s1 = np.random.normal(scale = 0.6, size=(300, ))
      try:
        s2 = glove[temp[1]]
      except KeyError:
        s2 = np.random.normal(scale = 0.6, size=(300, ))
      weights_matrix[i] = (s1 + s2) / 2
      final_vocab[word] = i
      continue
    weights_matrix[i] = glove[word]
    words_found += 1
  except KeyError:
    weights_matrix[i] = np.random.normal(scale = 0.6, size=(300, ))
  final_vocab[word] = i

In [0]:
class my_dataset(Dataset):
  def __init__(self,questions, answers, file_path, transform= None):
    self.path = file_path
    self.transforms = transform
    self.questions = questions
    self.answers = answers
    
  def __getitem__(self, item):
    current_question = self.questions[item]
    current_answer = self.answers[item]
    
    for i in range(len(current_question)):
      if 'image' in current_question[i]:
        index = copy.deepcopy(current_question[i])
        if index.replace('image','') == '':
          index = 'image1'
        current_question[i] = 'image'
    img_path = self.path + index +'.png'
    img = Image.open(img_path)
    if self.transforms is not None:
      img = self.transforms(img)
    
    designed_answer = []
    designed_answer.append(current_answer[0])
    sample = random.sample(current_question, 5)
    designed_answer.extend(sample)
    sample = random.sample(my_vocab, 26)
    designed_answer.extend(sample)
    
    ans_ind = [final_vocab[x] for x in designed_answer]
    ques_ind = [final_vocab[x] for x in current_question]
    length = len(ques_ind)
    while (len(ques_ind) < 31):
      ques_ind.append(0)
    
    return img, torch.tensor(ques_ind), torch.tensor(ans_ind), torch.tensor(length)
    
    
    
  def __len__(self):
    return len(self.questions)
batch_size = 32  
total_epoch = 20

In [0]:
class network(nn.Module):
  def __init__(self, matrix_weights):
    super(network, self).__init__()
    self.resnet = models.resnet18(pretrained = True)
    self.resnet.fc = nn.Sequential()
    self.linear_from_resnet_to_lstm = nn.Linear(512, 150)
    self.embedding, num_embeddings, embedding_dim = create_emb_layer(weights_matrix, True)
    self.gru = nn.GRU(embedding_dim, 150, 1, batch_first=True)
    self.fc1 = nn.Linear(662, 300)
    self.fc2 = nn.Linear(300, 300)
  
  def forward(self, image, question, answers, lengths):
    with torch.no_grad():
      image_features1 = self.resnet(image)
    image_features = self.linear_from_resnet_to_lstm(image_features1)

    image_features = image_features.unsqueeze(0)

    embedded = self.embedding(question)
    embedded = torch.nn.utils.rnn.pack_padded_sequence(embedded, batch_first=True, lengths = lengths, enforce_sorted =False)

    lstm_output = self.gru(embedded, image_features)

    concated = torch.cat((lstm_output[1].squeeze(0), image_features1), 1)

  
    output = self.fc1(concated)

    output = F.relu(output)
    output = self.fc2(output)


    answer_vec = self.embedding(answers)
    

    final_output = torch.einsum('bwf,bf->bw', answer_vec, output)
    
    return final_output
    

In [0]:
images_path = '/content/nyu_depth_images/'
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

test_length = 2500

my_data = my_dataset(all_questions, all_answers, images_path, trans)
data_range = range(len(my_data))
test_index = random.sample(range(len(my_data)),test_length)
train_index = [x for x in data_range if x not in test_index]

train_sampler = SubsetRandomSampler(train_index)
test_sampler = SubsetRandomSampler(test_index)



train_loader = torchdata.DataLoader(my_data, batch_size = batch_size, shuffle=False, sampler = train_sampler)
print(len(train_loader))
test_loader = torchdata.DataLoader(my_data, batch_size = batch_size, shuffle = False, sampler = test_sampler)
print(len(test_loader))

my_net = network(weights_matrix)
criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(my_net.parameters(), lr=0.01,weight_decay = 0.005)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
my_net.to(device)
optimizer.zero_grad()



In [0]:
all_loss = list()
test_loss = list()
for epoch in range(total_epoch):
  this_epoch_loss = list()
  this_epoch_test_loss= list()
  optimizer.zero_grad()
  my_net.train()
  for i, (image, question, answer, length) in enumerate(train_loader):
    image = image.to(device)
    question = question.to(device)
    answer = answer.to(device)
    image = Variable(image)
    question = Variable(question)
    answer = Variable(answer)
    optimizer.zero_grad()
    output = my_net(image, question, answer, length)
    label = torch.zeros([len(image)], dtype= torch.long)
    loss = criterion(output, label.to(device))
    loss.backward()
    this_epoch_loss.append(loss.item())
    print('in epoch {} and index {} loss is {}'.format(epoch, i , this_epoch_loss[-1]))
  print('in epoch {} average loss was {}'.format(epoch, np.mean(this_epoch_loss)))
  all_loss.append(np.mean(this_epoch_loss))
  
  my_net.eval()
  for i, (image, question, answer, length) in enumerate(test_loader):
    image = image.to(device)
    question = question.to(device)
    answer = answer.to(device)
    image = Variable(image)
    question = Variable(question)
    answer = Variable(answer)
    output = my_net(image, question, answer, length)
    label = torch.zeros([len(image)], dtype= torch.long)
    loss = criterion(output, label.to(device))
    this_epoch_test_loss.append(loss.item())
    print('in test epoch {} and index {} loss is {}'.format(epoch, i, this_epoch_test_loss[-1]))
  print('in test epoch {} average loss was {}'.format(epoch, np.mean(this_epoch_test_loss)))
  test_loss.append(np.mean(this_epoch_test_loss))
  

In [0]:
import matplotlib.pyplot as plt
fig = plt.figure()
plt.plot(all_loss,'r', label = 'train_loss')
plt.plot(test_loss,'b', label = 'test_loss')
plt.legend(loc='upper right')
plt.title('average loss in epoch')
plt.xlabel('# of epoch')
plt.ylabel('average Loss')
plt.show()